## Predict Reporter Sequences (Data-Validation)

I will use the gbm to predict the reporter sequences.

In [4]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
# my module imports
from optimalcodon.projects.rnastability.dataprocessing import get_data, general_preprocesing_pipeline

In [5]:
reporters = (
    pd.read_csv("../../19-03-13-PredictReportersWithModel/reporters.csv")
    .rename(columns={'sequence': 'coding'})
    .assign(
        gene_id = lambda x: x.reporter_id + '|' + x.optimality,
        utrlenlog = np.nan,
        cdslenlog = lambda x: np.log(x.coding.str.len()),
        key = 'k' # tmp var
    )
    .drop(['reporter_id', 'optimality', 'description'], axis=1)
)
reporters.head()

,coding,gene_id,utrlenlog,cdslenlog,key
0,ATGGTTTCAAAAGGAGAAGAAGATAATATGGCGATAATTAAAGAAT...,cherry-P2A-fish|optimal,NaN,7.334982,k
1,ATGGTTTCAAAAGGAGAAGAAGATAATATGGCGATAATTAAAGAAT...,cherry-P2A-fish|non-optimal,NaN,7.334982,k
2,ATGGCGAGAAGGTGTCTTCGTTTATGGCAACGGAGGCGTAGGAGCA...,embo2016-B|non-optimal,NaN,5.743003,k
3,ATGGCAGAAGGTGTCTTCGTTTATGGCAACGGAGGCGTAGGAGCAT...,embo2016-B|optimal,NaN,5.752573,k
4,ATGGTTTCAAAAGGAGAAGAAGATAATATGGCGATAATTAAAGAAT...,cherry-P2A-293t|optimal,NaN,7.334982,k


In [7]:
(train_x, train_y), (test_x, test_y) = get_data("../19-04-30-EDA/results_data/")
dtypefeaturs = (
    test_x[['specie', 'cell_type', 'datatype']]
    .drop_duplicates()
    .reset_index()
    .drop('gene_id', axis=1)
    .assign(key = 'k')
)


In [8]:
reporters = (
    pd.merge(reporters, dtypefeaturs, on='key')
    .drop('key', axis=1)
    .set_index('gene_id')
)
reporters.head()

,coding,utrlenlog,cdslenlog,specie,cell_type,datatype
gene_id,,,,,,
cherry-P2A-fish|optimal,ATGGTTTCAAAAGGAGAAGAAGATAATATGGCGATAATTAAAGAAT...,NaN,7.334982,human,k562,slam-seq
cherry-P2A-fish|optimal,ATGGTTTCAAAAGGAGAAGAAGATAATATGGCGATAATTAAAGAAT...,NaN,7.334982,human,k562,endogenous
cherry-P2A-fish|optimal,ATGGTTTCAAAAGGAGAAGAAGATAATATGGCGATAATTAAAGAAT...,NaN,7.334982,human,293t,endogenous
cherry-P2A-fish|optimal,ATGGTTTCAAAAGGAGAAGAAGATAATATGGCGATAATTAAAGAAT...,NaN,7.334982,human,hela,endogenous
cherry-P2A-fish|optimal,ATGGTTTCAAAAGGAGAAGAAGATAATATGGCGATAATTAAAGAAT...,NaN,7.334982,human,RPE,endogenous


## Data Pre-processing

In [9]:
print("{} points for training and {} for testing with {} features".format(
    train_x.shape[0], test_x.shape[0], test_x.shape[1]))

# pre-processing

preprocessing = general_preprocesing_pipeline(train_x)

preprocessing.fit(train_x)
train_x_transformed = preprocessing.transform(train_x)

train_x_transformed.shape

67817 points for training and 7534 for testing with 6 features


(67817, 80)

## Fit Algorithm

In [10]:
## fit gbm parameters found in grid search
gbm = GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='huber', max_depth=10,
             max_features='log2', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=3, min_samples_split=8,
             min_weight_fraction_leaf=0.0, n_estimators=2000,
             n_iter_no_change=None, presort='auto', random_state=None,
             subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)

gbm.fit(train_x_transformed, train_y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='huber', max_depth=10,
             max_features='log2', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=3, min_samples_split=8,
             min_weight_fraction_leaf=0.0, n_estimators=2000,
             n_iter_no_change=None, presort='auto', random_state=None,
             subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [11]:
reporters['predicted'] = gbm.predict(preprocessing.transform(reporters))

In [12]:
test_x['predicted'] = gbm.predict(preprocessing.transform(test_x))

In [13]:
reporters['observed'] = np.nan

In [14]:
test_x['observed'] = test_y
reporters

,coding,utrlenlog,cdslenlog,specie,cell_type,datatype,predicted,observed
gene_id,,,,,,,,
cherry-P2A-fish|optimal,ATGGTTTCAAAAGGAGAAGAAGATAATATGGCGATAATTAAAGAAT...,NaN,7.334982,human,k562,slam-seq,0.494301,NaN
cherry-P2A-fish|optimal,ATGGTTTCAAAAGGAGAAGAAGATAATATGGCGATAATTAAAGAAT...,NaN,7.334982,human,k562,endogenous,0.570095,NaN
cherry-P2A-fish|optimal,ATGGTTTCAAAAGGAGAAGAAGATAATATGGCGATAATTAAAGAAT...,NaN,7.334982,human,293t,endogenous,0.452106,NaN
cherry-P2A-fish|optimal,ATGGTTTCAAAAGGAGAAGAAGATAATATGGCGATAATTAAAGAAT...,NaN,7.334982,human,hela,endogenous,0.323277,NaN
cherry-P2A-fish|optimal,ATGGTTTCAAAAGGAGAAGAAGATAATATGGCGATAATTAAAGAAT...,NaN,7.334982,human,RPE,endogenous,0.307313,NaN
cherry-P2A-fish|optimal,ATGGTTTCAAAAGGAGAAGAAGATAATATGGCGATAATTAAAGAAT...,NaN,7.334982,fish,embryo mzt,aamanitin polya,0.263597,NaN
cherry-P2A-fish|optimal,ATGGTTTCAAAAGGAGAAGAAGATAATATGGCGATAATTAAAGAAT...,NaN,7.334982,fish,embryo mzt,aamanitin ribo,0.387326,NaN
cherry-P2A-fish|optimal,ATGGTTTCAAAAGGAGAAGAAGATAATATGGCGATAATTAAAGAAT...,NaN,7.334982,xenopus,embryo mzt,aamanitin ribo,0.413856,NaN
cherry-P2A-fish|optimal,ATGGTTTCAAAAGGAGAAGAAGATAATATGGCGATAATTAAAGAAT...,NaN,7.334982,mouse,mES cells,slam-seq,0.418063,NaN


In [15]:
(
    reporters
    .append(test_x)
    [['cell_type', 'datatype', 'specie', 'observed', 'predicted']]
    .to_csv('results_data/reporters_pr')
)


/home/smedina/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
